In [1]:
import os 

import numpy as np

import keras
import keras.backend as K
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, load_model
from keras.optimizers import Adam
import PIL
from keras.preprocessing.image import ImageDataGenerator

import scipy


import cv2
import datetime

In [2]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'D:\\Developer\\FaceMaskDetection\\Data\\train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'D:\\Developer\\FaceMaskDetection\\Data\\test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )
model.save('mymodel.h5',model_saved)

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\Aarti Jangra\AppData\Local\Temp\ipykernel_17616\2024784145.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
83/83 [==============================] - 25s 291ms/step - loss: 0.3484 - accuracy: 0.8607 - val_loss: 0.1437 - val_accuracy: 0.9588
Epoch 2/10
83/83 [==============================] - 35s 421ms/step - loss: 0.1556 - accuracy: 0.9521 - val_loss: 0.1117 - val_accuracy: 0.9588
Epoch 3/10
83/83 [==============================] - 30s 355ms/step - loss: 0.1258 - accuracy: 0.9574 - val_loss: 0.1170 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 22s 266ms/step - loss: 0.0952 - accuracy: 0.9680 - val_loss: 0.1036 - val_accuracy: 0.9691
Epoch 5/10
83/83 [==============================] - 23s 278ms/step - loss: 0.0979 - accuracy: 0.9680 - val_loss: 0.0815 - val_accuracy: 0.9691
Epoch 6/10
83/83 [==============================] - 24s 283ms/step - loss: 0.0975 - accuracy: 0.9635 - val_loss: 0.1018 - val_accuracy: 0.9691
Epoch 7/10
83/83 [==============================] - 23s 278ms/step - loss: 0.0964 - accuracy: 0.9665 - val_loss: 0.1953 - val_accuracy: 0.9433

d:\Developer\FaceMaskDetection\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import cv2
import numpy as np
import datetime
from keras.preprocessing import image
from keras.models import load_model

me_model=load_model('D:\\Developer\\FaceMaskDetection\\mymodel.h5')

face_cascade_path='D:\\Developer\\FaceMaskDetection\\Data\\haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)

if face_cascade.empty():
    print("error: unable to load face cascade")
    exit()

cap=cv2.VideoCapture(0)

if not cap.isOpened():
    print("error: unable to open camera")
    exit()

while True:
    ret , img=cap.read()
    if not ret:
        print("error: unable to read image")
        break

    face = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=4)

    for (x,y,w,h) in face:
        face_img=img[y:y+h,x:x+w]
        cv2.imwrite('temp.jpg',face_img)

        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)

        pred=me_model.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK', ((x+w)//2, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img, 'MASK', ((x+w)//2, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)

    cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 26ms/step


KeyboardInterrupt: 